# Langchain and Anthropic

Claude is chat-based model, meaning it is trained on conversation data. However, it is a text based API, meaning it takes in single string. It expects this string to be in a particular format. This means that it is up the user to ensure that is the case. LangChain provides several utilities and helper functions to make sure prompts that you write - whether formatted as a string or as a list of messages - end up formatted correctly.

Because Claude is chat-based but accepts a string as input, it can be treated as either a LangChain `ChatModel` or `LLM`. This means there are two wrappers in LangChain - ChatAnthropic and Anthropic. It is generally recommended to use the ChatAnthropic wrapper, and format your prompts as ChatMessages (we will show examples of this below). This is because it keeps your prompt in a general format that you can easily then also use with other models (should you want to). However, if you want more fine-grained control over the prompt, you can use the Anthropic wrapper - we will show and example of this as well. The `Anthropic` wrapper however is **deprecated**, as all functionality can be achieved in a more generic way using `ChatAnthropic`.

Ref: <https://python.langchain.com/docs/integrations/platforms/anthropic>

## Best Practices

#### No System Messages

Anthropic models are not trained on the concept of a "system message". We have worked with the Anthropic team to handle them somewhat appropriately (a Human message with an admin tag) but this is largely a hack and it is recommended that you do not use system messages.

#### AI Messages Can Continue

A completion from Claude is a continuation of the last text in the string which allows you further control over Claude's output.

For example, putting words in Claude's mouth in a prompt like this:

>`\n\nHuman: Tell me a joke about bears\n\nAssistant: What do you call a bear with no teeth?`

This will return a completion like this `A gummy bear!` instead of a whole new assistant message with a different random bear joke.

In [1]:
# !pip install -Uq boto3 anthropic-bedrock langchain rich

In [2]:
%load_ext rich

In [3]:
import re
import warnings
from pathlib import Path
from types import FunctionType

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT
from IPython.display import Markdown
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chat_models import ChatAnthropic
from langchain.llms.bedrock import Bedrock
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from rich import print
from utils import get_inference_parameters

warnings.filterwarnings("ignore")

## Setup

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [4]:
# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name="us-west-2")

bedrock_model_id = "anthropic.claude-v2"  # Bedrock model_id
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

bedrock_model = Bedrock(
    client=client, model_id=bedrock_model_id, model_kwargs=model_kwargs
)  # Initalize LLM

#### Creating Anthropic prompts with ChatPromptTemplate

When working with ChatModels, it is preferred that you design your prompts as `ChatPromptTemplates`.

Anthropic expects chat prompts to be formatted in a specific manner.

```text
\n\nHuman: Please translate the below text to french.\n\nAssistant:
```

We can chat messages by using the `human` and `assistant` prefixes when initializing the prompt. LangChain automatically adds the prefix and suffix.

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        ("human", "Hi! How are you?"),
        ("assistant", "I'm doing well. Thanks for asking."),
        ("human", "Tell me a joke about {topic}"),
    ]
)
print(prompt)

ChatPromptTemplate(
    input_variables=['topic'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')
        ),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hi! How are you?')),
        AIMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template="I'm doing well. Thanks for asking.")
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='Tell me a joke about {topic}')
        )
    ]
)

You can then invoke the chain like below:

*NOTE:* The below example is using Langchain Expression Language (LCEL)

Refer to <https://python.langchain.com/docs/expression_language> for more details.

In [6]:
chain = prompt | bedrock_model
chain.invoke({"topic": "cats"})

" Here's a silly joke about cats:\n\nWhy don't cats play poker in the jungle? Too many cheetahs!"

### Creating Prompts using `HumanMessage`

Below we use `HumanMessage` from `langchain.schema` to construct `messages` for the chat model.

We then create a `ChatPromptTemplate` from these messages to invoke this chain.


*NOTE: As there are no input variables to be passed, we send an empty `input` string*

In [7]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
prompt = ChatPromptTemplate.from_messages(messages=messages)

chain = prompt | bedrock_model
output = chain.invoke({"input": ""})

Markdown(output)

 Voici la traduction en français : J'adore programmer.

##### Another Variation

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "You're an AI assistant who's good at {ability}. Please answer this {question}",
        )
    ]
)
chain = prompt | bedrock_model

output = chain.invoke(
    {"ability": "computerscience", "question": "What are sorting algorithms"}
)

Markdown(output)

 Sorting algorithms are algorithms that put elements of a list in a certain order. Some common sorting algorithms include:

- Bubble sort - Repeatedly steps through a list, compares adjacent elements, and swaps them if they are in the wrong order.

- Insertion sort - Builds up a sorted list by inserting each new element into the correct position in the already sorted portion of the list.

- Selection sort - Finds the smallest remaining element and adds it to the end of the sorted list.

- Merge sort - Recursively divides a list into smaller sublists, sorts each sublist, then merges the sublists back together in sorted order. Uses a divide-and-conquer approach.

- Quicksort - Chooses a pivot element and partitions the list into two sublists - elements less than the pivot and elements greater than the pivot. Recursively sorts the sublists.

- Heapsort - Uses a binary heap data structure to sort the list. Builds a heap from the list, removes the largest element from the heap and places it at the end of the list, then reconstructs the heap with the remaining elements.

The choice of sorting algorithm depends on things like efficiency, memory usage, and stability. Some algorithms like merge sort and heapsort are faster but require more memory. Others like insertion sort are simple to implement but slower. The optimal algorithm depends on the specific use case.

#### Prompts with PromptTemplate

We can see that under the hood LangChain is not appending any prefix/suffix to SystemMessage's. This is because `Anthropic` has no concept of `SystemMessage`. 

Anthropic requires all prompts to end with assistant messages. This means if the last message is not an assistant message, the suffix `Assistant:` will automatically be inserted.

If you decide instead to use a normal `PromptTemplate` (one that just works on a single string) then we need do the following:

- Prefix our prompt string with `HUMAN_PROMPT` and suffix with `AI_PROMPT`

In [9]:
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
# Format prompt for Anthropic
anthropic_prompt = PromptTemplate.from_template(
    f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}"
)
print(anthropic_prompt)

# Invoke the Chain
chain = anthropic_prompt | bedrock_model
chain.invoke({"topic": "otters"})

PromptTemplate(input_variables=['topic'], template='\n\nHuman:Tell me a joke about {topic}\n\nAssistant:')

" Here's a silly joke about otters:\n\nWhat do you call an otter with a cold? A snotter!"

## Router Chain

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

For e.g., Say we have four prompts optimized for different types of questions, and we want to choose the prompt template based on the user input.


[!NOTE]: The below cells show a classical implementation of router chains. Implementing chains using LangChain Expression Language (LCEL) and Runnables is more intuitive and less verbose. Providing the below for comparision purposes.

Refer to LCEL implementation of Router Chain below in this notebook.

Reference: <https://python.langchain.com/docs/modules/chains/foundational/router>

In [10]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [11]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [12]:
llm = bedrock_model

In [13]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

#### Define a default_chain for inputs that doesn't match with any of the provided template.

In [14]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

#### Define the Multi-prompt Router prompt

In [15]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [17]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [18]:
output = chain.run(
    "List last 5 presidents of USA and their term durations. Output in a Markdown table format."
)

Markdown(output)



> Entering new MultiPromptChain chain...
History: {'input': 'List last 5 presidents of USA and their term durations. Output in a Markdown table format.'}
> Finished chain.


 Here is a list of the last 5 presidents of the USA and their term durations in a Markdown table format:

| President | Term |
|-|-|  
| Donald Trump | 2017 - 2021 |
| Barack Obama | 2009 - 2017 |
| George W. Bush | 2001 - 2009 |  
| Bill Clinton | 1993 - 2001 |
| George H.W. Bush | 1989 - 1993 |

## Router Chains using LCEL

There are two ways to perform routing:

1. Using a RunnableBranch.
2. Writing custom factory function that takes the input of a previous step and returns a **runnable**. Importantly, this should return a **runnable** and NOT actually execute.

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about Math, Physics, Computer Science or History, then routes to a corresponding prompt chain. We also define a *General* chain as a fall back mechanism.

Here, we create a simple `ROUTER_PROMPT_TEMPLATE` that classifies the input text into one of the above 4 categories.

In [19]:
llm = bedrock_model

ROUTER_PROMPT_TEMPLATE = """Given the user question below, classify it as either being about `math`, `physics`, `computerscience` or `history`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""

classify_chain = (
    PromptTemplate.from_template(template=ROUTER_PROMPT_TEMPLATE)
    | llm
    | StrOutputParser()
)

# test the chain
classify_chain.invoke(
    {"question": "List last 5 presidents of USA and their term durations."}
)

' history'

#### Create 4 chains - physics, history, math and computerscience

##### Physics chain to answer physics questions

In [20]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a <question> you admit\
that you don't know.

<question>
{question}
</question>

Answer:"""

physics_chain = PromptTemplate.from_template(template=physics_template) | llm

In [21]:
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

<question>
{question}
</question>

Answer:"""

math_chain = PromptTemplate.from_template(template=math_template) | llm

In [22]:
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

<question>
{question}
</question>

Answer:"""

history_chain = PromptTemplate.from_template(template=history_template) | llm

In [23]:
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

<question>
{question}
</question>

Answer:"""

cs_chain = PromptTemplate.from_template(template=computerscience_template) | llm

In [24]:
# General chain for fallback
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question: <question>{question}</question> Answer:"""
    )
    | llm
)

### Using a RunnableBranch

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Create a `RunnableBranch` that routes the input to the appropriate prompt.

In [25]:
prompt_branch = RunnableBranch(
    (lambda x: "math" in x["topic"].lower(), math_chain),
    (lambda x: "physics" in x["topic"].lower(), physics_chain),
    (lambda x: "history" in x["topic"].lower(), history_chain),
    (lambda x: "computerscience" in x["topic"].lower(), cs_chain),
    general_chain,
)

#### Putting it all together.

First, we pass the input **question** to a chain that categorizes, the **question** into one of 4 categories:

- Math
- Physics
- Computer Science
- History

Next, we pass the output to the `RunnableBranch`, which then, routes the **question** to the relevant *chain*

In [26]:
full_chain = {
    "topic": classify_chain,
    "question": lambda x: x["question"],
} | prompt_branch

In [27]:
# Check the input and output schema of the full_chain
print("Input schema: ", full_chain.input_schema.schema())
print("====" * 10)
print("Output schema: ", full_chain.output_schema.schema())

Input schema: 
{
    'title': 'RunnableParallelInput',
    'type': 'object',
    'properties': {'question': {'title': 'Question', 'type': 'string'}}
}

========================================

Output schema: 
{'title': 'RunnableBranchOutput'}

In [28]:
questions = [
    "List last 5 presidents of USA and their term durations. Output in a Markdown table format.",
    "What is 5% of 257",
    "What are black holes?",
    "How to implement Interfaces in Python?",
]

for question in questions:
    print(f"[b]Question:[/b] [i green]{question}[/i green]")
    output = full_chain.invoke({"question": question})
    display(Markdown(output))
    print()
    print("---" * 25)

Question: List last 5 presidents of USA and their term durations. Output in a Markdown table format.

 Here is a table of the last 5 presidents of the USA and their term durations:

| President | Term |
|-|-|  
| Joe Biden | 2021 - present |
| Donald Trump | 2017 - 2021 |
| Barack Obama | 2009 - 2017 |  
| George W. Bush | 2001 - 2009 |
| Bill Clinton | 1993 - 2001 |

---------------------------------------------------------------------------

Question: What is 5% of 257

 Okay, let's break this down step-by-step:

1) 5% means 5 out of 100
2) To find 5% of a number, we divide the number by 100 and multiply by 5
3) 257 / 100 = 2.57
4) 2.57 * 5 = 12.85

Therefore, 5% of 257 is 12.85.

...

 Black holes are regions of space with gravitational fields so strong that nothing, including light, can escape from within the black hole's event horizon. Some key facts about black holes:

- They form when massive stars collapse at the end of their life cycles. The gravitational collapse is so extreme that it causes a singularity, or a point of infinite density and space-time curvature. 

- They have an event horizon, which is the boundary beyond which nothing can escape the black hole's gravitational pull, not even light. The event horizon's size is proportional to the black hole's mass.

- They continue to grow in mass as they absorb surrounding material like gas, dust and stars that get too close and cross the event horizon. 

- They do not actively "suck" material in; objects get caught in their gravity once they come within a certain distance.

- They cannot be directly observed, but can be detected through their gravitational effects on nearby stars and gas. They also emit Hawking radiation.

- Matter that falls into a black hole is thought to be compressed into a singularity, but the laws of physics as we know them break down inside the event horizon.

- They are thought to exist at the center of most large galaxies, including our own Milky Way galaxy. The supermassive black hole at the Milky Way's center is called Sagittarius A*.

In summary, black holes are extremely dense regions of spacetime from which nothing can escape once it crosses the event horizon. They fascinate scientists and appear to play major roles in the evolution of galaxies.

---------------------------------------------------------------------------

Question: How to implement Interfaces in Python?

 Python does not have an interface construct like other object-oriented languages such as Java or C#. However, Python does allow for interfaces to be implemented through abstract base classes. Here is one way to implement interfaces in Python:

1. Create an abstract base class and use the abc module to make it an official abstract class:

```python
from abc import ABC, abstractmethod

class MyInterface(ABC):

    @abstractmethod
    def method1(self):
        pass

    @abstractmethod    
    def method2(self):
        pass
```

2. Any class that inherits from this abstract base class must implement all the abstract methods:

```python 
class MyClass(MyInterface):

    def method1(self):
        print("Implementing method1")

    def method2(self):
        print("Implementing method2")
```

3. Attempting to instantiate the abstract base class directly will raise an error:

```python
i = MyInterface() # Raises TypeError
```

4. But instantiating a concrete subclass will work:

```python
c = MyClass() 
c.method1()
c.method2()
```

So in summary, interfaces can be implemented by creating abstract base classes and having concrete classes inherit from them and implement the required methods. This allows for "interface-like" behavior in Python.

---------------------------------------------------------------------------